In [302]:
import plotly.graph_objects as go
import pandas as pd
import pandasql as pds
import random

In [303]:
data = pd.read_csv(r"C:\Users\MichaelDiFelice\Downloads\ind_cons_main_v2.csv")

data[['Maintenance', 'Induction', 'nextline_reg']] = data[['Maintenance', 'Induction', 'nextline_reg']].where(pd.notnull(data), 'None')

data.head()

,Unnamed: 0,PatientID,LineNumber,Induction,Consolidation,Maintenance,NextLine_Number,nextline_start,nextline_reg
0,0,F001E7DD3C229,1.0,"Dexamethasone,Lenalidomide",No Consolidation,Lenalidomide,2.0,NaN,None
1,1,F001FEAEFA6AB,2.0,"Clinical Study Drug,Lenalidomide",No Consolidation,Lenalidomide,3.0,2023-07-18,Daratumumab
2,2,F002651CEF65E,1.0,Lenalidomide,No Consolidation,Lenalidomide,2.0,2022-09-30,"Carfilzomib,Daratumumab,Dexamethasone"
3,3,F0056ED626E36,1.0,"Bortezomib,Cyclophosphamide,Dexamethasone,Lena...",No Consolidation,None,2.0,NaN,None
4,4,F0057E24E0E55,1.0,"Bortezomib,Dexamethasone,Lenalidomide",No Consolidation,Bortezomib,2.0,2020-09-03,"Daratumumab,Dexamethasone,Pomalidomide"


In [304]:
# Function to apply SQL query using pandasql
def pysqldf(q):
    return pds.sqldf(q, globals())

In [305]:
induction_sql = '''
SELECT *,
CASE
    WHEN Induction LIKE '%Carfilzomib%,%Cyclophosphamide%,%Dexamethasone%,%Thalidomide%' THEN 'KCTd'
    WHEN Induction LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DVRd'
    WHEN Induction LIKE '%Carfilzomib%,%Dexamethasone%,%Lenalidomide%,%Pomalidomide%' THEN 'KPRd'
    WHEN Induction LIKE '%Carfilzomib%,%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DKRd'
    WHEN Induction LIKE '%Bortezomib%,%Daratumumab%,%Melphalan%,%Prednisone%' THEN 'DVMp'
    WHEN Induction LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%,%Thalidomide%' THEN 'DVTd'
    WHEN Induction LIKE '%Bortezomib%,%Dexamethasone%,%Pomalidomide%' THEN 'PVd'
    WHEN Induction LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%' THEN 'DVd'
    WHEN Induction LIKE '%Bortezomib%,%Dexamethasone%,%Lenalidomide%' THEN 'VRd'
    WHEN Induction LIKE '%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DRd'
    WHEN Induction LIKE '%Bortezomib%,%Cyclophosphamide%,%Dexamethasone%' THEN 'VCd'
    WHEN Induction LIKE '%Dexamethasone%,%Ixazomib%,%Lenalidomide%' THEN 'IRd'
    WHEN Induction LIKE '%Daratumumab%,%Dexamethasone%,%Pomalidomide%' THEN 'DPd'
    WHEN Induction LIKE '%Carfilzomib%,%Dexamethasone%,%Lenalidomide%' THEN 'KRd'
    WHEN Induction LIKE '%Carfilzomib%,%examethasone%,%Pomalidomide%' THEN 'KPd'
    WHEN Induction LIKE '%Dexamethasone%,%Elotuzumab%,Pomalidomide%' THEN 'EPd'
    WHEN Induction LIKE '%Carfilzomib%,%Daratumumab%,%Dexamethasone%' THEN 'DKd'
    WHEN Induction LIKE '%Carfilzomib%,%Dexamethasone%,%Isatuximab-Irfc%' THEN 'IsaKd'
    WHEN Induction LIKE '%Bortezomib%,%Dexamethasone%,%Thalidomide%' THEN 'VTd'
    WHEN Induction LIKE '%Elotuzumab%,%Dexamethasone%,%Lenalidomide%' THEN 'ERd'
    WHEN Induction LIKE '%Dexamethasone%,%Isatuximab-Irfc%,%Pomalidomide%' THEN 'IsaPd'
    WHEN Induction LIKE '%Cyclophosphamide%,%Dexamethasone%,%Pomalidomide%' THEN 'PCd'
    WHEN Induction LIKE '%Cyclophosphamide%,%Dexamethasone%' THEN 'Cd'
    WHEN Induction LIKE '%Carfilzomib%,%Dexamethasone%' THEN 'Kd'
    WHEN Induction LIKE '%Dexamethasone%,%Lenalidomide%' THEN 'Rd'
    WHEN Induction LIKE '%Bortezomib%,%Dexamethasone%' THEN 'Vd'
    WHEN Induction LIKE '%Bortezomib%,%Lenalidomide' THEN 'VR'
    WHEN Induction LIKE '%Dexamethasone%,Pomalidomide%' THEN 'Pd'
    WHEN Induction LIKE '%Daratumumab%,%Dexamethasone%' THEN 'Dd'
    WHEN Induction LIKE '%Transplant%' THEN 'SCT'
    WHEN Induction LIKE '%Daratumumab%' THEN 'D Mono'
    WHEN Induction LIKE '%Ixazomib%' THEN 'Other Ixa'
    WHEN Induction LIKE '%Isatuximab-Irfc%' THEN 'Other Isa'
    WHEN Induction LIKE '%Ciltacabtagene Autoleucel%' THEN 'CAR-T'
    WHEN Induction LIKE 'Bortezomib' THEN 'V mono'
    WHEN Induction LIKE '%Selinexor%' THEN 'Selinexor'
    WHEN Induction LIKE 'Lenalidomide' THEN 'R mono'
    WHEN Induction LIKE 'Dexamethasone' THEN 'd mono'
    WHEN Induction LIKE 'Carfilzomib' THEN 'K mono'
    WHEN Induction LIKE 'Pomalidomide' THEN 'P mono'
    WHEN Induction LIKE '%Cyclophosphamide%' THEN 'C mono'
    WHEN Induction LIKE '%Clinical Study Drug%' THEN 'Clinical Study Drug'
    WHEN Induction LIKE '%None%' THEN 'None'
    ELSE 'Other'
END AS induction_regimen
FROM data;
'''


In [306]:
next_line_sql = '''
SELECT *,
CASE
    WHEN nextline_reg LIKE '%Carfilzomib%,%Cyclophosphamide%,%Dexamethasone%,%Thalidomide%' THEN 'KCTd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DVRd'
    WHEN nextline_reg LIKE '%Carfilzomib%,%Dexamethasone%,%Lenalidomide%,%Pomalidomide%' THEN 'KPRd'
    WHEN nextline_reg LIKE '%Carfilzomib%,%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DKRd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Daratumumab%,%Melphalan%,%Prednisone%' THEN 'DVMp'
    WHEN nextline_reg LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%,%Thalidomide%' THEN 'DVTd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Dexamethasone%,%Pomalidomide%' THEN 'PVd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%' THEN 'DVd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Daratumumab%,%Lenalidomide%' THEN 'DVR'
    WHEN nextline_reg LIKE '%Bortezomib%,%Dexamethasone%,%Lenalidomide%' THEN 'VRd'
    WHEN nextline_reg LIKE '%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DRd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Cyclophosphamide%,%Dexamethasone%' THEN 'VCd'
    WHEN nextline_reg LIKE '%Dexamethasone%,%Ixazomib%,%Lenalidomide%' THEN 'IRd'
    WHEN nextline_reg LIKE '%Daratumumab%,%Dexamethasone%,%Pomalidomide%' THEN 'DPd'
    WHEN nextline_reg LIKE '%Carfilzomib%,%Dexamethasone%,%Lenalidomide%' THEN 'KRd'
    WHEN nextline_reg LIKE '%Carfilzomib%,%examethasone%,%Pomalidomide%' THEN 'KPd'
    WHEN nextline_reg LIKE '%Dexamethasone%,%Elotuzumab%,Pomalidomide%' THEN 'EPd'
    WHEN nextline_reg LIKE '%Carfilzomib%,%Daratumumab%,%Dexamethasone%' THEN 'DKd'
    WHEN nextline_reg LIKE '%Carfilzomib%,%Dexamethasone%,%Isatuximab-Irfc%' THEN 'IsaKd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Dexamethasone%,%Thalidomide%' THEN 'VTd'
    WHEN nextline_reg LIKE '%Elotuzumab%,%Dexamethasone%,%Lenalidomide%' THEN 'ERd'
    WHEN nextline_reg LIKE '%Dexamethasone%,%Isatuximab-Irfc%,%Pomalidomide%' THEN 'IsaPd'
    WHEN nextline_reg LIKE '%Cyclophosphamide%,%Dexamethasone%,%Pomalidomide%' THEN 'PCd'
    WHEN nextline_reg LIKE '%Cyclophosphamide%,%Dexamethasone%' THEN 'Cd'
    WHEN nextline_reg LIKE '%Carfilzomib%,%Dexamethasone%' THEN 'Kd'
    WHEN nextline_reg LIKE '%Dexamethasone%,%Lenalidomide%' THEN 'Rd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Dexamethasone%' THEN 'Vd'
    WHEN nextline_reg LIKE '%Bortezomib%,%Lenalidomide' THEN 'VR'
    WHEN nextline_reg LIKE '%Dexamethasone%,Pomalidomide%' THEN 'Pd'
    WHEN nextline_reg LIKE '%Daratumumab%,%Dexamethasone%' THEN 'Dd'
    WHEN nextline_reg LIKE '%Transplant%' THEN 'SCT'
    WHEN nextline_reg LIKE '%Daratumumab%' THEN 'D Mono'
    WHEN nextline_reg LIKE '%Ixazomib%' THEN 'Other Ixa'
    WHEN nextline_reg LIKE '%Isatuximab-Irfc%' THEN 'Other Isa'
    WHEN nextline_reg LIKE '%Ciltacabtagene Autoleucel%' THEN 'CAR-T'
    WHEN nextline_reg LIKE 'Bortezomib' THEN 'V mono'
    WHEN nextline_reg LIKE '%Selinexor%' THEN 'Selinexor'
    WHEN nextline_reg LIKE 'Lenalidomide' THEN 'R mono'
    WHEN nextline_reg LIKE 'Dexamethasone' THEN 'd mono'
    WHEN nextline_reg LIKE 'Carfilzomib' THEN 'K mono'
    WHEN nextline_reg LIKE 'Pomalidomide' THEN 'P mono'
    WHEN nextline_reg LIKE '%Cyclophosphamide%' THEN 'C mono'
    WHEN nextline_reg LIKE '%Clinical Study Drug%' THEN 'Clinical Study Drug'
    WHEN nextline_reg LIKE '%None%' THEN 'None'
    ELSE 'Other'
END AS next_line_regimen
FROM data;
'''


In [307]:
maintenance_sql = '''
SELECT *,
CASE
    WHEN Maintenance LIKE '%Carfilzomib%,%Cyclophosphamide%,%Dexamethasone%,%Thalidomide%' THEN 'KCTd'
    WHEN Maintenance LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DVRd'
    WHEN Maintenance LIKE '%Carfilzomib%,%Dexamethasone%,%Lenalidomide%,%Pomalidomide%' THEN 'KPRd'
    WHEN Maintenance LIKE '%Carfilzomib%,%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DKRd'
    WHEN Maintenance LIKE '%Bortezomib%,%Daratumumab%,%Melphalan%,%Prednisone%' THEN 'DVMp'
    WHEN Maintenance LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%,%Thalidomide%' THEN 'DVTd'
    WHEN Maintenance LIKE '%Bortezomib%,%Dexamethasone%,%Pomalidomide%' THEN 'PVd'
    WHEN Maintenance LIKE '%Bortezomib%,%Daratumumab%,%Dexamethasone%' THEN 'DVd'
    WHEN Maintenance LIKE '%Bortezomib%,%Dexamethasone%,%Lenalidomide%' THEN 'VRd'
    WHEN Maintenance LIKE '%Daratumumab%,%Dexamethasone%,%Lenalidomide%' THEN 'DRd'
    WHEN Maintenance LIKE '%Bortezomib%,%Cyclophosphamide%,%Dexamethasone%' THEN 'VCd'
    WHEN Maintenance LIKE '%Dexamethasone%,%Ixazomib%,%Lenalidomide%' THEN 'IRd'
    WHEN Maintenance LIKE '%Daratumumab%,%Dexamethasone%,%Pomalidomide%' THEN 'DPd'
    WHEN Maintenance LIKE '%Carfilzomib%,%Dexamethasone%,%Lenalidomide%' THEN 'KRd'
    WHEN Maintenance LIKE '%Carfilzomib%,%examethasone%,%Pomalidomide%' THEN 'KPd'
    WHEN Maintenance LIKE '%Dexamethasone%,%Elotuzumab%,Pomalidomide%' THEN 'EPd'
    WHEN Maintenance LIKE '%Carfilzomib%,%Daratumumab%,%Dexamethasone%' THEN 'DKd'
    WHEN Maintenance LIKE '%Carfilzomib%,%Dexamethasone%,%Isatuximab-Irfc%' THEN 'IsaKd'
    WHEN Maintenance LIKE '%Bortezomib%,%Dexamethasone%,%Thalidomide%' THEN 'VTd'
    WHEN Maintenance LIKE '%Elotuzumab%,%Dexamethasone%,%Lenalidomide%' THEN 'ERd'
    WHEN Maintenance LIKE '%Dexamethasone%,%Isatuximab-Irfc%,%Pomalidomide%' THEN 'IsaPd'
    WHEN Maintenance LIKE '%Cyclophosphamide%,%Dexamethasone%,%Pomalidomide%' THEN 'PCd'
    WHEN Maintenance LIKE '%Cyclophosphamide%,%Dexamethasone%' THEN 'Cd'
    WHEN Maintenance LIKE '%Carfilzomib%,%Dexamethasone%' THEN 'Kd'
    WHEN Maintenance LIKE '%Dexamethasone%,%Lenalidomide%' THEN 'Rd'
    WHEN Maintenance LIKE '%Bortezomib%,%Dexamethasone%' THEN 'Vd'
    WHEN Maintenance LIKE '%Bortezomib%,%Lenalidomide' THEN 'VR'
    WHEN Maintenance LIKE '%Dexamethasone%,Pomalidomide%' THEN 'Pd'
    WHEN Maintenance LIKE '%Daratumumab%,%Dexamethasone%' THEN 'Dd'
    WHEN Maintenance LIKE '%Transplant%' THEN 'SCT'
    WHEN Maintenance LIKE '%Daratumumab%' THEN 'D Mono'
    WHEN Maintenance LIKE '%Ixazomib%' THEN 'Other Ixa'
    WHEN Maintenance LIKE '%Isatuximab-Irfc%' THEN 'Other Isa'
    WHEN Maintenance LIKE '%Ciltacabtagene Autoleucel%' THEN 'CAR-T'
    WHEN Maintenance LIKE 'Bortezomib' THEN 'V mono'
    WHEN Maintenance LIKE '%Selinexor%' THEN 'Selinexor'
    WHEN Maintenance LIKE 'Lenalidomide' THEN 'R mono'
    WHEN Maintenance LIKE 'Dexamethasone' THEN 'd mono'
    WHEN Maintenance LIKE 'Carfilzomib' THEN 'K mono'
    WHEN Maintenance LIKE 'Pomalidomide' THEN 'P mono'
    WHEN Maintenance LIKE '%Cyclophosphamide%' THEN 'C mono'
    WHEN Maintenance LIKE '%Clinical Study Drug%' THEN 'Clinical Study Drug'
    WHEN Maintenance LIKE '%None%' THEN 'None'
    ELSE 'Other'
END AS maintenance_regimen
FROM data;
'''

In [308]:
#  Figure out how to deal with NULL Values

data = pds.sqldf(maintenance_sql, locals())
data = pds.sqldf(next_line_sql, locals())
data = pds.sqldf(induction_sql, locals())


data.dropna(subset=['nextline_reg'], inplace=True)
# data = data.drop(columns=['induction_regimen'])


In [309]:
data.head()

,Unnamed: 0,PatientID,LineNumber,Induction,Consolidation,Maintenance,NextLine_Number,nextline_start,nextline_reg,maintenance_regimen,next_line_regimen,induction_regimen
0,0,F001E7DD3C229,1.0,"Dexamethasone,Lenalidomide",No Consolidation,Lenalidomide,2.0,None,None,R mono,None,Rd
1,1,F001FEAEFA6AB,2.0,"Clinical Study Drug,Lenalidomide",No Consolidation,Lenalidomide,3.0,2023-07-18,Daratumumab,R mono,D Mono,Clinical Study Drug
2,2,F002651CEF65E,1.0,Lenalidomide,No Consolidation,Lenalidomide,2.0,2022-09-30,"Carfilzomib,Daratumumab,Dexamethasone",R mono,DKd,R mono
3,3,F0056ED626E36,1.0,"Bortezomib,Cyclophosphamide,Dexamethasone,Lena...",No Consolidation,None,2.0,None,None,None,None,VRd
4,4,F0057E24E0E55,1.0,"Bortezomib,Dexamethasone,Lenalidomide",No Consolidation,Bortezomib,2.0,2020-09-03,"Daratumumab,Dexamethasone,Pomalidomide",V mono,DPd,VRd


In [310]:
data = data[(data['nextline_start'] > '2022-08-01') & (data['NextLine_Number'] == 2)]


In [311]:
data = data[data['nextline_reg'].str.contains('Daratumumab') | data['nextline_reg'].str.contains('Isatuximab-Irfc')]


In [312]:
def bundle_regimen(row,column,line):
    # Line-based conditions
    if row[line] == 1:
        # Transplant and regimen conditions within Line 1
        if row[column] == "DVRd":
            return "DVRd"
        elif row[column] == "DVTd":
            return "DVTd"
        elif row[column] == "VCd":
            return "VCd"
        elif row[column] in ["VRd", "VR", "Vd",]:
            return "VRd"
        elif row[column] in ['D mono','DKd','DKRd','DPd','DVd','DVMp','DVTd','Other D']:
            return "D-other"

    elif row[line] == 2:
        # Transplant and regimen conditions within Line 2
        if row[column] in ['D mono','DKRd','DVMp','DVRd','DVTd','Other D']:
            return "D-other"
        elif row[column] == "DKd":
            return "DKd"
        elif row[column] == "DPd":
            return "DPd"
        elif row[column] == "DRd":
            return "DRd"
        elif row[column] == "DVd":
            return "DVd"
        elif row[column] == "IsaKd":
            return "IsaKd"
        elif row[column] == "IsaPd":
            return "IsaPd"
        elif row[column] == 'Rd':
            return 'Rd'
        elif row[column] in ['ERd','IRd','KRd','VRd']:
            return 'R-Triplet'
    elif row[line] == 3:
        # Transplant and regimen conditions within Line 2
        if row[column] in ['D mono','DKRd','DVMp','DVRd','DVTd','Other D']:
            return "D-other"
        elif row[column] == "DKd":
            return "DKd"
        elif row[column] == "DPd":
            return "DPd"
        elif row[column] == "DRd":
            return "DRd"
        elif row[column] == "DVd":
            return "DVd"
        elif row[column] == "IsaKd":
            return "IsaKd"
        elif row[column] == "IsaPd":
            return "IsaPd"
        elif row[column] == 'Rd':
            return 'Rd'
        elif row[column] in ['ERd','IRd','KRd','VRd']:
            return 'R-Triplet'


    # Default return value if none of the conditions are met
    return "Other"




In [313]:
data.head()

,Unnamed: 0,PatientID,LineNumber,Induction,Consolidation,Maintenance,NextLine_Number,nextline_start,nextline_reg,maintenance_regimen,next_line_regimen,induction_regimen
2,2,F002651CEF65E,1.0,Lenalidomide,No Consolidation,Lenalidomide,2.0,2022-09-30,"Carfilzomib,Daratumumab,Dexamethasone",R mono,DKd,R mono
35,35,F027680413E60,1.0,"Bortezomib,Dexamethasone,Lenalidomide",No Consolidation,Lenalidomide,2.0,2023-01-17,"Daratumumab,Dexamethasone,Pomalidomide",R mono,DPd,VRd
59,59,F04374188F8E8,1.0,"Bortezomib,Dexamethasone,Lenalidomide",No Consolidation,Lenalidomide,2.0,2022-11-17,"Daratumumab,Dexamethasone,Pomalidomide",R mono,DPd,VRd
63,63,F0495AD1C46B7,1.0,"Bortezomib,Dexamethasone,Lenalidomide",No Consolidation,Lenalidomide,2.0,2022-08-24,"Azacitidine,Carfilzomib,Daratumumab,Dexamethasone",R mono,DKd,VRd
138,138,F0BF76C2F2769,1.0,"Bortezomib,Dexamethasone,Lenalidomide",No Consolidation,Lenalidomide,2.0,2022-08-26,"Carfilzomib,Daratumumab,Dexamethasone",R mono,DKd,VRd


In [314]:
data['induction_regimen'] = data.apply(bundle_regimen,args=('induction_regimen','LineNumber',), axis=1)
data['next_line_regimen'] = data.apply(bundle_regimen,args=('next_line_regimen','NextLine_Number',), axis=1)

In [343]:

# Helper function to add identifier to regimen names
def add_identifier(regimen, identifier):
    return regimen + "_" + identifier

# Grouping data to get counts for each flow
induction_counts = data['induction_regimen'].value_counts()
consolidation_counts = data['Consolidation'].value_counts()
maintenance_counts = data['maintenance_regimen'].value_counts()
line2_counts = data['next_line_regimen'].value_counts()

# Counting unique patients for each combination of Induction, Maintenance, and Line 2 regimens
combination_counts = data.groupby(['induction_regimen','Consolidation','maintenance_regimen', 'next_line_regimen']).size().reset_index(name='count')

# Initializing the lists for the Sankey diagram
source = []
target = []
value = []
label = []

# Populating the lists for Induction to Maintenance connections
for index, row in combination_counts.iterrows():
    # Induction
    if add_identifier(row['induction_regimen'], "I") not in label:
        label.append(add_identifier(row['induction_regimen'], "I"))
    source.append(label.index(add_identifier(row['induction_regimen'], "I")))
    
    # Maintenance
    if add_identifier(row['maintenance_regimen'], "M") not in label:
        label.append(add_identifier(row['maintenance_regimen'], "M"))
    target.append(label.index(add_identifier(row['maintenance_regimen'], "M")))
    
    # Value
    value.append(row['count'])
    

# Populating the lists for Maintenance to Line 2 connections
for index, row in combination_counts.iterrows():
    # Maintenance
    if add_identifier(row['maintenance_regimen'], "M") not in label:
        label.append(add_identifier(row['maintenance_regimen'], "M"))
    source.append(label.index(add_identifier(row['maintenance_regimen'], "M")))
    
    # Line 2
    if add_identifier(row['next_line_regimen'], "2") not in label:
        label.append(add_identifier(row['next_line_regimen'], "2"))
    target.append(label.index(add_identifier(row['next_line_regimen'], "2")))
    
    # Value
    value.append(row['count'])

# Removing the identifiers for node labels
label_clean = [l.split("_")[0] for l in label]

# Generate a random color
def generate_random_color():
    return 'rgba({}, {}, {}, 1)'.format(random.randint(0, 255), 
                                        random.randint(0, 255), 
                                        random.randint(0, 255))

# Create a color map with random colors for each unique node label
color_map = {label: generate_random_color() for label in label_clean}

# Determine the colors for nodes based on the color_map
node_colors = [color_map[label] for label in label_clean]

# Use a loop to determine the color of each link based on its source node
link_colors = [node_colors[src] for src in source]

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=label_clean,
        color=node_colors
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=link_colors
    )
))

fig.update_layout(
    height=600,  # Adjust the height
    width=1000,   # Adjust the width
    font_size=18  # Adjust the font size
)
fig

In [349]:
7/75

0.09333333333333334